## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop additional fields to attempt to increase accuracy
application_df = application_df.drop(columns=['EIN', 'NAME', 'USE_CASE', 'STATUS', 'SPECIAL_CONSIDERATIONS'])

In [3]:
# Determine the number of unique values in each column.
unique = application_df.nunique()
print(unique)

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(app_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [5]:
# Update to 1100 for attempted accuracy increase
# use the variable name `application_types_to_replace`
cutoff = 1100  
application_types_to_replace = app_type_counts[app_type_counts < cutoff].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     3331
T4        1542
T6        1216
T5        1173
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [7]:
# Updated cutoff to 1900
# use the variable name `classifications_to_replace`
cutoff_value_class = 1900
classifications_to_replace = classification_counts[classification_counts <= cutoff_value_class].index.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     4144
C3000     1918
Name: count, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE','AFFILIATION', 'CLASSIFICATION', 'ORGANIZATION', 'INCOME_AMT',])


In [9]:
# Change random_state to 10 Split our preprocessed data into our features and target arrays
y=application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, stratify=y)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Attempt 1 Compile, Train and Evaluate the Model

In [11]:
# 1 Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
layer_1 = 90
layer_2 = 50

input_features = X_train.shape[1]

nn_1 = tf.keras.models.Sequential()

# First hidden layer
nn_1.add(tf.keras.layers.Dense(units=layer_1, input_dim=input_features, activation="relu"))

# Second hidden layer
nn_1.add(tf.keras.layers.Dense(units=layer_2, activation="relu"))

# Output layer
nn_1.add(tf.keras.layers.Dense(units= 1, activation="sigmoid"))

# Check the structure of the model
nn_1.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 90)                2790      
                                                                 
 dense_1 (Dense)             (None, 50)                4550      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 7391 (28.87 KB)
Trainable params: 7391 (28.87 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# 1 Compile the model
nn_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# 1 Train the model
fit_model = nn_1.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50


804/804 [==============================] - 3s 2ms/step - loss: 0.5861 - accuracy: 0.7095
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5760 - accuracy: 0.7147
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5735 - accuracy: 0.7167
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5728 - accuracy: 0.7149
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5717 - accuracy: 0.7157
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5704 - accuracy: 0.7170
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5707 - accuracy: 0.7171
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5701 - accuracy: 0.7170
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5698 - accuracy: 0.7171
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5699 - accuracy: 0.71

In [14]:
# 1 Evaluate the model using the test data
model_loss_1, model_accuracy_1 = nn_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_1}, Accuracy: {model_accuracy_1}")

268/268 - 1s - loss: 0.5746 - accuracy: 0.7207 - 1s/epoch - 4ms/step
Loss: 0.574646532535553, Accuracy: 0.7206997275352478


In [15]:
# Export our model to HDF5 file
nn_1.save("AlphabetSoupCharity_1.h5")

c:\Users\tpuch\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Attempt 2 Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
layer_1 = 90
layer_2 = 50
layer_3 = 40
input_features = X_train.shape[1]

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=layer_1, input_dim=input_features, activation="relu"))


# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=layer_2, activation="relu"))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=layer_3, activation="relu"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units= 1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_1"


_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 90)                2790      
                                                                 
 dense_4 (Dense)             (None, 50)                4550      
                                                                 
 dense_5 (Dense)             (None, 40)                2040      
                                                                 
 dense_6 (Dense)             (None, 1)                 41        
                                                                 
Total params: 9421 (36.80 KB)
Trainable params: 9421 (36.80 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Compile the model using Stochastic Gradient Descent for attempt 2
nn_2.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [18]:
# 2 Train the model
fit_model = nn_2.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 5s 4ms/step - loss: 0.6293 - accuracy: 0.6756
Epoch 2/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5843 - accuracy: 0.7097
Epoch 3/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5781 - accuracy: 0.7136
Epoch 4/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5754 - accuracy: 0.7145
Epoch 5/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5739 - accuracy: 0.7149
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5726 - accuracy: 0.7150
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5721 - accuracy: 0.7158
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5712 - accuracy: 0.7158
Epoch 9/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5708 - accuracy: 0.7165
Epoch 10/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5704 - accuracy: 0.7177

In [19]:
# 2 Evaluate the model using the test data
model_loss_2, model_accuracy_2 = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Optimization 2 - Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")

268/268 - 1s - loss: 0.5719 - accuracy: 0.7202 - 764ms/epoch - 3ms/step
Optimization 2 - Loss: 0.5718878507614136, Accuracy: 0.7202332615852356


In [21]:
# Export our model to HDF5 file
nn_2.save("AlphabetSoupCharity_2.h5")

c:\Users\tpuch\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Attempt 3 Compile, Train and Evaluate the Model

In [22]:
#3 Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
layer_1 = 128
layer_2 = 90
layer_3 = 50
layer_4 = 25
input_features = X_train.shape[1]

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=layer_1, input_dim=input_features, activation="relu"))


# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=layer_2, activation="relu"))

# Third hidden layer
nn_3.add(tf.keras.layers.Dense(units=layer_3, activation="ELU"))

# Fourth hidden layer
nn_3.add(tf.keras.layers.Dense(units=layer_4, activation="ELU"))

# Output layer
nn_3.add(tf.keras.layers.Dense(units= 1, activation="sigmoid"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 128)               3968      
                                                                 
 dense_8 (Dense)             (None, 90)                11610     
                                                                 
 dense_9 (Dense)             (None, 50)                4550      
                                                                 
 dense_10 (Dense)            (None, 25)                1275      
                                                                 
 dense_11 (Dense)            (None, 1)                 26        
                                                                 
Total params: 21429 (83.71 KB)
Trainable params: 21429 (83.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# 3 Compile the model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# 3 Train the model
fit_model = nn_3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100


804/804 [==============================] - 7s 5ms/step - loss: 0.5842 - accuracy: 0.7065
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5760 - accuracy: 0.7135
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5739 - accuracy: 0.7144
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5736 - accuracy: 0.7158
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5718 - accuracy: 0.7135
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5715 - accuracy: 0.7172
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5711 - accuracy: 0.7161
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5708 - accuracy: 0.7168
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5703 - accuracy: 0.7154
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5699 - accuracy: 0.7179
E

In [25]:
# 3 Evaluate the model using the test data
model_loss_3, model_accuracy_3 = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Optimization 3 - Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

268/268 - 1s - loss: 0.5805 - accuracy: 0.7190 - 738ms/epoch - 3ms/step
Optimization 3 - Loss: 0.5805099606513977, Accuracy: 0.7189504504203796


In [26]:
# Export our model to HDF5 file
nn_3.save("AlphabetSoupCharity_3.h5")

### Compare 3 Optomization Attempts


In [27]:
print(f"Optimization 1 - Loss: {model_loss_1}, Accuracy: {model_accuracy_1}")
print(f"Optimization 2 - Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")
print(f"Optimization 3 - Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

Optimization 1 - Loss: 0.574646532535553, Accuracy: 0.7206997275352478
Optimization 2 - Loss: 0.5718878507614136, Accuracy: 0.7202332615852356
Optimization 3 - Loss: 0.5805099606513977, Accuracy: 0.7189504504203796
